In [2]:
import sys, os
import pickle
import multiprocessing as mp
import numpy as np
import pandas as pd
from src.neuron import *
from src.utils import *
from src.constants import * 
from src.network import *
from src.validation import *
from src.viz import *
from src.genetic_algorithm import *


In [24]:
# List of CSV files
csvs = [
"/Users/stevenwendel/Documents/GitHub/bg/minimized_data_pass_0.csv",
"/Users/stevenwendel/Documents/GitHub/bg/minimized_data_pass_1.csv", 
"/Users/stevenwendel/Documents/GitHub/bg/minimized_data_pass_2.csv",
"/Users/stevenwendel/Documents/GitHub/bg/minimized_data_pass_3.csv",
"/Users/stevenwendel/Documents/GitHub/bg/minimized_data_pass_4.csv",
"/Users/stevenwendel/Documents/GitHub/bg/minimized_data_pass_5.csv",
"/Users/stevenwendel/Documents/GitHub/bg/minimized_data_pass_6.csv"
]
dfs = []
for csv in csvs:
    try:
        df_temp = pd.read_csv(csv)
        if not df_temp.empty:
            # Convert dna string to list and dna_score to int
            df_temp['dna'] = df_temp['dna'].apply(eval)  # Converts string representation of list to actual list
            df_temp['dna_score'] = df_temp['dna_score'].astype(int)
            dfs.append(df_temp)
    except pd.errors.EmptyDataError:
        print(f"Warning: {csv} is empty and will be skipped.")

df = pd.concat(dfs, ignore_index=True)

# Remove duplicates, sort by score, and reset index
df = df.drop_duplicates(subset='dna').sort_values('dna_score', ascending=False).reset_index(drop=True)

In [25]:
display(df)
print(df.iloc[0])
print(len(df.iloc[0]['dna']))
print(type(df.iloc[0]['dna']))

,dna,dna_score
0,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741
1,"[1000, 0, 0, 888, -32, -8, 0, 0, -27, 0, 0, 0,...",741
2,"[1000, 0, 0, 888, -32, -8, 0, 0, -27, 0, 0, 0,...",741
3,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741
4,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741
...,...,...
4933,"[1000, 0, 0, 1000, -5, 0, 0, 0, -15, 0, 0, 0, ...",731
4934,"[1000, 0, 0, 1000, -5, 0, 0, 0, -15, 0, 0, 0, ...",731
4935,"[1000, 0, 0, 1000, -5, 0, 0, 0, -15, 0, 0, 0, ...",731
4936,"[99, 0, 0, 291, -118, 0, 0, 0, -58, 0, 0, 0, -...",731


dna          [1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...
dna_score                                                  741
Name: 0, dtype: object
49
<class 'list'>


In [45]:
# [1000, 0 , 0, 80, 20, 6,...] and [1000, 0 , 0, 543, 20, 6,...] are considered the same
# This takes the top dna as a representative of each unique configuration
max_synapses = 20

# Dictionary to store unique configurations and their details
unique_configs = {}
for index, row in df.iterrows():
    config = tuple(1 if weight != 0 else 0 for weight in row['dna'])
    # Calculate the number of non-zero entries
    non_zero_count = sum(config)
    # Only consider configurations with non-zero counts of 18 or lower
    if non_zero_count <= max_synapses and config not in unique_configs:
        # Store the index, dna_score, and non_zero count
        unique_configs[config] = {
            'index': index,
            'dna_score': row['dna_score'],
            'non_zero_count': non_zero_count
        }

# Create a list of indices and non_zero counts for the unique configurations
unique_indices = [details['index'] for details in unique_configs.values()]
non_zero_counts = [details['non_zero_count'] for details in unique_configs.values()]

# Create a new dataframe with only the unique configurations
unique_df = df.loc[unique_indices].copy().reset_index(drop=True)
unique_df['non_zero_count'] = non_zero_counts

print(f"Found {len(unique_df)} unique configurations")
display(unique_df)

Found 1104 unique configurations


,dna,dna_score,non_zero_count
0,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741,18
1,"[1000, 0, 0, 888, -32, -8, 0, 0, -27, 0, 0, 0,...",741,18
2,"[1000, 0, 0, 888, -32, -8, 0, 0, -27, 0, 0, 0,...",741,17
3,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741,17
4,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741,17
...,...,...,...
1099,"[1000, 0, 0, 1000, 0, 0, 0, 0, -15, 0, 0, 0, -...",731,18
1100,"[1000, 0, 0, 1000, 0, -891, 0, 0, -15, 0, 0, 0...",731,18
1101,"[1000, 0, 0, 1000, 0, -891, 0, 0, -15, 0, 0, 0...",731,18
1102,"[1000, 0, 0, 1000, 0, -891, 0, 0, -15, 0, 0, 0...",731,18


In [46]:
# Recalculating scores just in case...
scores = []
unique_df2 = unique_df.copy()
for i in range(len(unique_df2)):
    print(f'Processing Index: {i}')
    curr_dna = unique_df2.iloc[i]['dna']
   
    # Load DNA
    dna_matrix = load_dna(curr_dna)

    # === Preparing Network === 
    all_neurons = create_neurons()
    splits, input_waves, alpha_array = create_experiment()
    criteria_dict = define_criteria()
    max_score = TMAX//BIN_SIZE * len(CRITERIA_NAMES)

    dna_score, neuron_data = evaluate_dna(
                    dna_matrix=dna_matrix,
                    neurons=all_neurons,
                    alpha_array=alpha_array,
                    input_waves=input_waves,
                    criteria=criteria_dict,
                    curr_dna=curr_dna
                    )
    total_score = sum(dna_score.values())
    scores.append(total_score)
    
    print(f'    === DNA: {curr_dna}') 
    print(f'    === Control: {dna_score["control"]}/{max_score}')
    print(f'    === Experimental: {dna_score["experimental"]}/{max_score}')
    print(f'    === Overall: {total_score}({total_score/(2*max_score):.2%})')
    print('\n')
    

    diagnostic = {
            'show_dna_matrix' : False,
            'show_neuron_plots' : False,
            'show_difference_histogram' : False,
            'show_dna_scores': False,
            'show_spike_bins': False
        }

    if diagnostic['show_dna_matrix']:
                    print("Currently loaded matrix ---")
                    display_matrix(dna_matrix, NEURON_NAMES)

    if diagnostic['show_dna_scores']:
                    print(f'{dna_score=}: {curr_dna}')

    if diagnostic['show_spike_bins']:
        bin_size = 1000
        for condition in ['experimental', 'control']:
            print(f"Condition: {condition}")
            for neuron_name, data in neuron_data[condition].items():
                spike_times = data['spike_times']
                num_bins = len(spike_times) // bin_size
                spikes_per_bin = [int(np.sum(spike_times[i*bin_size:(i+1)*bin_size])) for i in range(num_bins)]                
                print(f"Neuron: {neuron_name}, {spikes_per_bin}")
                
    if diagnostic['show_neuron_plots']:
                    for condition in ['experimental', 'control']:
                        target_neurons_hist_Vs = np.array([neuron_data[condition][name]['hist_V'] for name in NEURON_NAMES])
                        plot_neurons_interactive(hist_Vs=target_neurons_hist_Vs, neuron_names=NEURON_NAMES, sq_wave=input_waves[0], go_wave=input_waves[1], show_u=False)

# Add scores as a new column to unique_df2                        
unique_df2['updated_score'] = scores

Processing Index: 0
    === DNA: [1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, -36, 0, 0, 0, -708, 0, 0, 46, 0, 0, 0, 0, 950, 397, 0, 0, 0, 0, 0, 411, 101, 525, 0, 0, 0, 0, -16, 0, 0, 0, -180, -105, 0, 0, 42, 632, 229]
    === Control: 249/500
    === Experimental: 492/500
    === Overall: 741(74.10%)


Processing Index: 1
    === DNA: [1000, 0, 0, 888, -32, -8, 0, 0, -27, 0, 0, 0, -36, 0, 0, 0, -708, 0, 0, 46, 0, 0, 0, 0, 950, 397, 0, 0, 0, 0, 0, 411, 101, 525, 0, 0, 0, 0, 0, 0, 0, 0, -180, -105, 0, 0, 42, 632, 229]
    === Control: 249/500
    === Experimental: 492/500
    === Overall: 741(74.10%)


Processing Index: 2
    === DNA: [1000, 0, 0, 888, -32, -8, 0, 0, -27, 0, 0, 0, -36, 0, 0, 0, -708, 0, 0, 46, 0, 0, 0, 0, 950, 397, 0, 0, 0, 0, 0, 411, 101, 525, 0, 0, 0, 0, 0, 0, 0, 0, 0, -105, 0, 0, 42, 632, 229]
    === Control: 249/500
    === Experimental: 492/500
    === Overall: 741(74.10%)


Processing Index: 3
    === DNA: [1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, -36, 0, 0, 

In [48]:
display(unique_df2)
# Replace dna_score with updated_score values
unique_df2['dna_score'] = unique_df2['updated_score']

# Drop the updated_score column
unique_df2.drop('updated_score', axis=1, inplace=True)

score_threshold = 730
unique_df2 = unique_df2[unique_df2['dna_score'] >= score_threshold].copy().sort_values('dna_score', ascending=False).reset_index(drop=True)
display(unique_df2)

,dna,dna_score,non_zero_count
0,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741,18
1,"[1000, 0, 0, 888, -32, -8, 0, 0, -27, 0, 0, 0,...",741,17
2,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741,17
3,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741,17
4,"[1000, 0, 0, 888, -32, 0, 0, 0, -27, 0, 0, 0, ...",741,16
...,...,...,...
1030,"[207, 0, 0, 443, -15, 0, 0, 0, -108, 0, 0, 0, ...",730,15
1031,"[207, 0, 0, 443, -15, 0, 0, 0, -108, 0, 0, 0, ...",730,15
1032,"[207, 0, 0, 443, -15, 0, 0, 0, -108, 0, 0, 0, ...",730,16
1033,"[207, 0, 0, 443, -15, 0, 0, 0, -108, 0, 0, 0, ...",730,16


KeyError: 'updated_score'

In [ ]:
# Optional:Check the matrix and plots for single DNA

curr_dna = unique_df2.iloc[1034]['dna']
# Load DNA
dna_matrix = load_dna(curr_dna)


# === Preparing Network === 
all_neurons = create_neurons()
splits, input_waves, alpha_array = create_experiment()
criteria_dict = define_criteria()
max_score = TMAX//BIN_SIZE * len(CRITERIA_NAMES)

dna_score, neuron_data = evaluate_dna(
                dna_matrix=dna_matrix,
                neurons=all_neurons,
                alpha_array=alpha_array,
                input_waves=input_waves,
                criteria=criteria_dict,
                curr_dna=curr_dna
                )
total_score = sum(dna_score.values())
print(f'    === DNA: {curr_dna}') 
print(f'    === Control: {dna_score["control"]}/{max_score}')
print(f'    === Experimental: {dna_score["experimental"]}/{max_score}')
print(f'    === Overall: {total_score}({total_score/(2*max_score):.2%})')
print('\n')

# STRUCTURE OF NEURON_DATA
# neuron_data = {
#     'experimental': {
#         'neuron_name': {
#             'hist_V': np.array(),
#             'spike_times': np.array()
#         },
#     'control': {
#         'neuron_name': {
#             'hist_V': np.array(),
#             'spike_times': np.array()
#             }
#         }
#     }
# }


diagnostic = {
        'show_dna_matrix' : True,
        'show_neuron_plots' : True,
        'show_difference_histogram' : False,
        'show_dna_scores': False
    }


if diagnostic['show_dna_matrix']:
                print("Currently loaded matrix ---")
                display_matrix(dna_matrix, NEURON_NAMES)

if diagnostic['show_dna_scores']:
                print(f'{dna_score=}: {curr_dna}')
            
if diagnostic['show_neuron_plots']:
                for condition in ['experimental', 'control']:
                    target_neurons_hist_Vs = np.array([neuron_data[condition][name]['hist_V'] for name in NEURON_NAMES])
                    plot_neurons_interactive(hist_Vs=target_neurons_hist_Vs, neuron_names=NEURON_NAMES, sq_wave=input_waves[0], go_wave=input_waves[1], show_u=False)
                    

In [75]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider
import pandas as pd
# Sort the DataFrame by dna_score (descending) and non_zero_count (ascending)
dg_df = unique_df2.sort_values(['dna_score', 'non_zero_count'], ascending=[False, True])

# Assuming unique_df3 is already defined and contains the necessary data
# Ensure ACTIVE_SYNAPSES and NEURON_NAMES are defined in your context

# Calculate the number of DNAs
num_dnas = len(dg_df)

# Determine which connections are shared by more than 50% of the DNAs
shared_connections = np.zeros(len(ACTIVE_SYNAPSES), dtype=int)

for dna in dg_df['dna']:
    shared_connections += np.array([1 if weight != 0 else 0 for weight in dna])

# Identify connections shared by more than 50% of the DNAs
shared_threshold = num_dnas * 0.5
shared_indices = [i for i, count in enumerate(shared_connections) if count > shared_threshold]

# Calculate proportion for each shared connection
shared_proportions = {i: shared_connections[i]/num_dnas for i in shared_indices}

# Function to determine the width of each edge
def get_edge_width(weight):
    return np.ceil(abs(weight) / 200) + 1

# Function to determine the style of each edge
def get_edge_style(weight):
    return 'dashed' if weight < 0 else 'solid'

# Function to plot a specific configuration
def plot_configuration(config_index):
    details = dg_df.iloc[config_index]
    dna = details['dna']
    print(f"Configuration Index: {config_index}, \n Score: {details['dna_score']} \n Non-zero count: {details['non_zero_count']}")
    
    # Create a labeled Series for the DNA
    best_run_labeled = pd.Series(dna, index=[f"{pair[0]} -> {pair[1]}" for pair in ACTIVE_SYNAPSES])

    # Create a graph
    G = nx.DiGraph()

    # Add nodes for each neuron
    G.add_nodes_from(NEURON_NAMES)

    # Add edges for non-zero synaptic connections
    for synapse, weight in best_run_labeled.items():
        if weight != 0:
            source, target = synapse.split(' -> ')
            G.add_edge(source, target, weight=weight)

    # Define custom positions for each node
    custom_pos = {
        'Somat': (5, 8.5),
        'MSN1': (5, 6),
        'MSN2': (3, 6),
        'MSN3': (0, 6),
        'SNR1': (5, 2.5),
        'SNR2': (3, 2.5),
        'SNR3': (0, 2.5),
        'ALMinter': (2, 8.5),
        'PPN': (2, 0),
        'THALgo': (2, 4.5),
        'VMprep': (4, 1),
        'ALMprep': (4, 7),
        'ALMresp': (1, 7),
        'VMresp': (1, 1)
    }

    # Draw the main graph using the custom positions
    plt.figure(figsize=(10, 6))
    edges = G.edges(data=True)
    nx.draw(G, pos=custom_pos, with_labels=True, node_size=2000, node_color='lightblue', font_size=10, font_weight='bold', arrowsize=20,
            width=[get_edge_width(data['weight']) for _, _, data in edges],
            style=[get_edge_style(data['weight']) for _, _, data in edges],
            edge_color='black')  # Default edge color

    # Create offset positions for shared connections
    offset_pos = custom_pos.copy()
    offset = 0.18  # Adjust this value to control the offset amount
    
    # Add shared connections in red with offset and arrows
    for index in shared_indices:
        source, target = ACTIVE_SYNAPSES[index]
        # Calculate offset positions
        source_pos = np.array(custom_pos[source])
        target_pos = np.array(custom_pos[target])
        # Add perpendicular offset
        direction = target_pos - source_pos
        perpendicular = np.array([-direction[1], direction[0]])
        perpendicular = perpendicular / np.linalg.norm(perpendicular) * offset
        
        # Shrink the arrow length by 20%
        midpoint = source_pos + 0.5 * direction
        source_pos_adjusted = midpoint - 0.4 * direction  # Move start point 10% closer to midpoint
        target_pos_adjusted = midpoint + 0.4 * direction  # Move end point 10% closer to midpoint
        
        offset_pos[source] = tuple(source_pos_adjusted + perpendicular)
        offset_pos[target] = tuple(target_pos_adjusted + perpendicular)
        
        # Draw red arrows with arrowheads and transparency
        nx.draw_networkx_edges(G, pos=offset_pos, edgelist=[(source, target)], 
                             edge_color='red', width=2, arrows=True, arrowsize=20, alpha=0.1)
        
        # Add proportion label to the red edges
        edge_labels = {(source, target): f'{shared_proportions[index]:.0%}'}
        nx.draw_networkx_edge_labels(G, pos=offset_pos, edge_labels=edge_labels, 
                                   font_color='red', font_size=8, alpha=0.5)

    # Highlight non-shared connections in blue
    non_shared_edges = [(source, target) for i, (source, target) in enumerate(ACTIVE_SYNAPSES) if i not in shared_indices and G.has_edge(source, target)]
    nx.draw_networkx_edges(G, pos=custom_pos, edgelist=non_shared_edges, edge_color='blue', width=2, style='dashed')

    # Add edge labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos=custom_pos, edge_labels=edge_labels, font_color='gray',font_size=8)

    plt.title('Synaptic Connections: Shared (Red) vs Non-Shared (Blue)')
    plt.show()

# Create an interactive slider to flip through the configurations
interact(plot_configuration, config_index=IntSlider(min=0, max=len(dg_df)-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='config_index', max=1034), Output()), _dom_classes=('widg…

<function __main__.plot_configuration(config_index)>